In [1]:
import pandas as pd
from e11 import H5Scan
from bokeh.io import output_notebook, show,curdoc
from bokeh.plotting import figure,ColumnDataSource
from bokeh.models import Panel, Tabs,Slider,TextInput
from bokeh.layouts import row, widgetbox

from functions import *
import pickle
output_notebook();
path = '../visualisation/data/';year = '2019';month = '08';day='14';
file_nums = ['002','003']
dfs = load_data(year,month,day,file_nums,path)
sources = []
ave_points=3
for data in dfs:
    plot_frame = pd.DataFrame()
    plot_frame['variable'] = data['v0']
    plot_frame['signal'] = -(data['a1']-data['a0'])
    plot_frame['background'] = data['a2']-data['a0']
    plot_frame['rolling_signal'] = plot_frame['signal'].rolling(ave_points).mean()
    plot_frame['rolling_background'] = plot_frame['background'].rolling(ave_points).mean()
    sources.append(ColumnDataSource(plot_frame))

Loading BokehJS ...

In [2]:
p1 = figure(plot_width=800, plot_height=400,tooltips = TOOLTIPS)
p2 = figure(x_range=p1.x_range,y_range=p1.y_range,plot_width=800, plot_height=400,tooltips = TOOLTIPS)

tab1 = Panel(child=p1, title="signal")
tab2 = Panel(child=p2, title="averaged")
tabs = Tabs(tabs=[ tab1, tab2 ])

for i,s in enumerate(sources):
    p1.line(x='variable', y='signal', source=s,color=plot_colors[i],legend='file {}'.format(file_nums[i]))
    p2.line(x='variable', y='rolling_signal',source=s,color=plot_colors[i],legend='file {}'.format(file_nums[i]))
p1.legend.click_policy = "hide"
p2.legend.click_policy = "hide"
show(tabs)